In [1]:
import numpy as np
import dms_estimator as dmse
from matplotlib import pyplot as plt

%matplotlib inline
plt.rc('figure', dpi=100)
np.set_printoptions(linewidth=100, suppress=True, precision=2, floatmode='fixed')

In [2]:
# Pyridine
from ipywidgets import interact, IntSlider, Dropdown

cfg = dmse.ModelConfig(
        name="Pyridine",
        build_integrator=dmse.pyridine_problem,
        true_p=[1.81, 0.894, 29.4, 9.21, 0.058, 2.43, 0.0644, 5.55, 0.0201, 0.577, 2.15],
        x0=np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        p_init=[1.0] * 11,
        state_labels=list("ABCDEFG"),
)

def run_estimation(num_meas, num_shooting, strategy):
    print(f"Start with (num_meas={num_meas}, num_shooting={num_shooting}, strategy='{strategy}'):")
    T = 5.0
    dt = T / num_meas
    t_grid = np.arange(0.0, T, dt)

    integrator, ode, states, params = cfg.build_integrator(dt)
    _, X_meas = dmse.generate_data(integrator, t_grid, cfg.x0, cfg.true_p, noise_std=1e-2)
    # Perform estimation
    p_hat, s_hat = dmse.estimate(
        ode,
        states,
        params,
        t_grid,
        X_meas,
        cfg.p_init,
        num_shooting=num_shooting,
        strategy=strategy,
    )
    # Plot results
    dmse.plot(t_grid, X_meas, s_hat, cfg.state_labels, show_every=5)
    plt.show()
    print(p_hat)
    print("True parameters:")
    print(np.array(cfg.true_p))

# Create interactive sliders and dropdown
_ = interact(
    run_estimation,
    num_meas=IntSlider(
        value=500,
        min=1,
        max=500,
        step=1,
        description='#Meas',
        continuous_update=False
    ),
    num_shooting=IntSlider(
        value=50,
        min=1,
        max=500,
        step=1,
        description='#Node',
        continuous_update=False
    ),
    strategy=Dropdown(
        options=['ipopt', 'gn', 'gn_fast'],
        value='gn',
        description='Strategy'
    )
)

interactive(children=(IntSlider(value=500, continuous_update=False, description='#Meas', max=500, min=1), IntS…